<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Testing-of-Code" data-toc-modified-id="Testing-of-Code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Testing of Code</a></span></li><li><span><a href="#Fit-the-Particle-Filter" data-toc-modified-id="Fit-the-Particle-Filter-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fit the Particle Filter</a></span></li></ul></div>

# Testing of Code

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import distributions as dist
from collections import Counter
from scipy.optimize import minimize, Bounds
import math

In [2]:
from functools import partial
from multiprocessing import Pool

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from models import particle as ptcl

In [5]:
run = 0
stims, labels = ptcl.get_zmstims(400)
types = "cc"
Cparam = 0.65
mu0 = np.mean(stims, 0)
sigma0 = np.var(stims, 0)
lambda0 = np.ones(len(stims[0]))
a0 = np.ones(len(stims[0]))
n_labels = 4
feedback = [1, 2, 3, 1, 2, 2, 2, 4, 3, 1]

args = [Cparam, mu0, sigma0, lambda0, a0, types, n_labels, feedback]

model = ptcl.RationalParticle(args, decision = "MAP")

In [6]:
model.N = 1
model.partition = [0]

In [7]:
model.labelprob()

array([0.25, 0.25, 0.25, 0.25])

In [8]:
model.N = 2
model.partition = [0, 0]

In [9]:
model.labelprob()

array([[0.4 , 0.2 , 0.2 , 0.2 ],
       [0.25, 0.25, 0.25, 0.25]])

# Fit the Particle Filter

In [10]:
import pandas as pd

In [11]:
tbl_train = pd.read_csv("data/tbl_train.csv")
tbl_transfer = pd.read_csv("data/tbl_transfer.csv")

In [12]:
stimuli = tbl_train[tbl_train["participant"] == 102][["d1i", "d2i"]].iloc[0:100, ]
feedback = tbl_train[tbl_train["participant"] == 102]["category"].iloc[0:100]
# use integer labels
feedback.replace({"A":1, "B":2, "C":3}, inplace=True)

fit only c parameter, keep others fixed

start_params = np.array([.5])

bounds = Bounds([0], [1])

def wrap_optim(id, tbl_data):
    stimuli_subset = tbl_data[tbl_data["participant"] == id][["d1i", "d2i"]]
    feedback_subset = tbl_data[tbl_data["participant"] == id]["category"]
    res = minimize(
        ptcl.fit_ll, start_params, 
        args = (stimuli_subset, feedback_subset), 
        method = "trust-constr", bounds=bounds,
        options={'gtol': 1e-6, 'disp': True}
    )
    l_out = [id, res]
    return l_out

f_partial = partial(wrap_optim, stimuli=tbl_train, feedback=feedback)
p = multiprocessing.Pool(2)
list_result = []
for x in p.imap(f_partial, participants):
    list_result.append(x)

In [13]:
participants = tbl_train["participant"].unique()[0:2]

def wrap_fit_particle(id, tbl_data):
    stimuli_subset = tbl_data[tbl_data["participant"] == id][["d1i", "d2i"]].to_numpy()
    feedback_subset = tbl_data[tbl_data["participant"] == id]["category"].to_numpy()
    c_seq = np.seq(0.025, 0.975, by=.05)
    lls = list()
    for c in c_seq:
        lls.append(fit_ll(c, stimuli_subset, feedback_subset))
    return lls

wrap_fit_partial = partial(ptcl.grid_search_particle, tbl_data = tbl_train)

In [14]:
list_to_process_no_names = [group for name, group in tbl_train.groupby("participant")]

In [15]:
tbl_train["trial_id"] = tbl_train.groupby("participant").cumcount()+1

In [16]:
tbl_train_small = tbl_train[tbl_train["trial_id"] < 50].copy()

In [18]:
l_results = ptcl.multiprocessing_grid_search_particle(tbl_train_small)

100%|████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 34.34it/s]


In [ ]:
p1 = tbl_train[tbl_train["participant"] == 101]

In [ ]:
p1 = p1.iloc[0:100,:].copy()


In [ ]:
ptcl.grid_search_particle(p1)

In [ ]:
l_results = list(map(ptcl.grid_search_particle, list_to_process_no_names))

In [ ]:
l_results

In [ ]:
l_results = ptcl.multiprocessing_grid_search_particle(tbl_train)

optimize particle filter
- takes too long, though

In [ ]:
res = minimize(
    ptcl.fit_ll, start_params, 
    args = (stimuli_np, feedback_np), 
    method = "trust-constr", bounds=bounds,
    options={'gtol': 1e-6, 'disp': True}
)

In [ ]:
res

In [ ]:
participants.tolist()

In [ ]:
l_results = list(map(ptcl.grid_search_particle, list_to_process_no_names[0:2]))